![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

ill save my data as some new dataframes and see what i have.

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')
qbr_df = pd.read_csv('data/qbr_season_level.csv') 

ill start with stats_df as that will require more work

In [3]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,0.292378,0,0.0,0.052632,NaN,NaN,0,12.7,13.7
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,1,0.377009,0,0.0,0.117647,NaN,NaN,0,5.1,8.1
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,-0.699578,0,NaN,0.023810,NaN,NaN,0,0.2,0.2
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,-0.228454,0,0.0,0.050000,NaN,NaN,0,3.5,5.5
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,3.9,3.9


In [4]:
# lets check up on Aaron Rodgers
stats_df[stats_df['player_display_name'] == 'Aaron Rodgers'].head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
34491,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.00,0.00
34492,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,15,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-2.60,-2.60
34493,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,17,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-0.10,-0.10
38709,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.56,0.56
38710,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,11,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.38,0.38


In [5]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129739 entries, 0 to 129738
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_id                    129739 non-null  object 
 1   player_name                  61504 non-null   object 
 2   player_display_name          129736 non-null  object 
 3   position                     129667 non-null  object 
 4   position_group               129667 non-null  object 
 5   headshot_url                 69803 non-null   object 
 6   recent_team                  129739 non-null  object 
 7   season                       129739 non-null  int64  
 8   week                         129739 non-null  int64  
 9   season_type                  129739 non-null  object 
 10  opponent_team                129739 non-null  object 
 11  completions                  129739 non-null  int64  
 12  attempts                     129739 non-null  int64  
 13 

In [6]:
# Use 1 to show all info in cell below then comment it out and use 2 to go back to default view I can delete this later

#1
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#2
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')


In [7]:
stats_df['position'].value_counts()

WR     49363
RB     33350
TE     24147
QB     16086
FB      5219
P        459
CB       148
G        128
DB       123
T         94
FS        93
SS        77
OLB       56
LS        56
K         46
LB        45
ILB       32
C         28
DE        28
DT        20
OT        17
DL        15
HB        13
NT        10
MLB        8
OG         3
OL         2
S          1
Name: position, dtype: int64

ive only got offensive related stats going back to 1999. I wont be needing those older players in a team for today

In [8]:
#lets look at the last ten years of data 
stats_df = stats_df[stats_df['season'].isin(range(2014, 2024))]

In [9]:
#lets only look at the regular season so we can see every player in action. this is about players not teams
stats_df = stats_df[stats_df['season_type'] == 'REG']


again these are offensive stats but they do give me a few key pieces of info that i can credit the oppenent team with. the opponent of an offense being.....a defense!!

In [36]:
# Define a function to handle grouping, summing, renaming, and filling NaNs
def process_stat(stat_df, stat_name, new_name=None):
    if new_name is None:
        new_name = stat_name
    grouped_data = stat_df.groupby(['opponent_team', 'season'])[stat_name].sum().reset_index()
    grouped_data.rename(columns={'opponent_team': 'defense', stat_name: new_name}, inplace=True)
    grouped_data.fillna(0, inplace=True)
    return grouped_data

# List of statistics to process
stats_to_process = [
    ('interceptions', 'interceptions_total'),
    ('sacks', 'sacks'),
    ('sack_yards', 'sack_yards'),
    ('receiving_yards', 'receiving_yards'),
    ('rushing_yards', 'rushing_yards'),
    ('rushing_tds', 'rushing_tds'),
    ('receiving_tds', 'receiving_tds'),
    ('sack_fumbles', 'sack_fumbles'),
    ('rushing_fumbles', 'rushing_fumbles'),
    ('receiving_fumbles', 'receiving_fumbles'),
    ('sack_fumbles_lost', 'sack_fumbles_recovered'),
    ('rushing_fumbles_lost', 'rushing_fumbles_recovered'),
    ('receiving_fumbles_lost', 'receiving_fumbles_recovered')
]

# Process each stat and store in a list of DataFrames
processed_stats = [process_stat(stats_df, stat, new_name) for stat, new_name in stats_to_process]

# Merge all DataFrames on 'defense' and 'season'
defense_df = processed_stats[0]
for df in processed_stats[1:]:
    defense_df = pd.merge(defense_df, df, on=['defense', 'season'], how='outer')

# Display the resulting DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2


In [27]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   defense                            320 non-null    object
 1   season                             320 non-null    int64 
 2   interceptions_total                320 non-null    int64 
 3   sacks_total                        320 non-null    int64 
 4   sack_yards_total                   320 non-null    int64 
 5   rushing_yards_total                320 non-null    int64 
 6   receiving_yards_total              320 non-null    int64 
 7   rushing_tds_total                  320 non-null    int64 
 8   receiving_tds_total                320 non-null    int64 
 9   sack_fumbles_total                 320 non-null    int64 
 10  rushing_fumbles_total              320 non-null    int64 
 11  receiving_fumbles                  320 non-null    int64 
 12  sack_fum

just need to create some ratios and totals for defense and that should be ready 

In [41]:
# Create forced_fumbles column
defense_df['forced_fumbles'] = (defense_df['sack_fumbles'] +
                                defense_df['rushing_fumbles'] +
                                defense_df['receiving_fumbles']
                               )
# Create fumbles_recovered column
defense_df['fumbles_recovered'] = (defense_df['sack_fumbles_recovered'] +
                                   defense_df['rushing_fumbles_recovered'] +
                                   defense_df['receiving_fumbles_recovered']
                                  )

# Calculate the percentage of forced fumbles that were fumbles recovered
defense_df['percentage_fumbles_recovered'] = (defense_df['fumbles_recovered'] / defense_df['forced_fumbles'] * 100).fillna(0)

# Create tds_against column
defense_df['tds_against'] = (defense_df['rushing_tds'] +
                             defense_df['receiving_tds']
                            )

#Create yards_against column
defense_df['yards_against'] = (defense_df['rushing_yards'] +
                             defense_df['receiving_yards'] -
                             defense_df['sack_yards']
                            )

# Display the updated DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered,forced_fumbles,fumbles_recovered,perecentage_fumbles_recovered,tds_against,yards_against,good_season,percentage_fumbles_recovered
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4,10,6,60.000000,31,5891,False,60.000000
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2,24,12,50.000000,33,5147,False,50.000000
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1,25,11,44.000000,37,4883,False,44.000000
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2,17,5,29.411765,36,4975,False,29.411765
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2,15,9,60.000000,46,5741,True,60.000000


In [38]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   defense                        320 non-null    object 
 1   season                         320 non-null    int64  
 2   interceptions_total            320 non-null    int64  
 3   sacks                          320 non-null    int64  
 4   sack_yards                     320 non-null    int64  
 5   receiving_yards                320 non-null    int64  
 6   rushing_yards                  320 non-null    int64  
 7   rushing_tds                    320 non-null    int64  
 8   receiving_tds                  320 non-null    int64  
 9   sack_fumbles                   320 non-null    int64  
 10  rushing_fumbles                320 non-null    int64  
 11  receiving_fumbles              320 non-null    int64  
 12  sack_fumbles_recovered         320 non-null    int

gotta scale and get features in order 

In [42]:
# Define the Target
score_median = defense_df['tds_against'].median()
defense_df['good_season'] = defense_df['tds_against'] > score_median

# Define features and target
features = defense_df.drop(columns=['defense', 'good_season', 'percentage_fumbles_recovered'])
target = defense_df['good_season'].astype(int)  # Convert boolean to int

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [43]:
# Create a dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# Fit the dummy classifier
dummy_clf.fit(X_train, y_train)

# Predict using the dummy classifier
y_pred_dummy = dummy_clf.predict(X_test)

# Calculate accuracy
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
report_dummy = classification_report(y_test, y_pred_dummy)

print("Dummy Classifier Accuracy:", accuracy_dummy)
print("Dummy Classifier Report:\n", report_dummy)

Dummy Classifier Accuracy: 0.5416666666666666
Dummy Classifier Report:
               precision    recall  f1-score   support

           0       0.54      1.00      0.70        52
           1       0.00      0.00      0.00        44

    accuracy                           0.54        96
   macro avg       0.27      0.50      0.35        96
weighted avg       0.29      0.54      0.38        96



c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
# Train a RandomForest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        52
           1       1.00      1.00      1.00        44

    accuracy                           1.00        96
   macro avg       1.00      1.00      1.00        96
weighted avg       1.00      1.00      1.00        96



In [ ]:
#make qb_stats_df
#rushers_stats_df
#receivers_stats_df

In [ ]:
#drop unnesscessary columns from each df

### Feature Engineering

In [ ]:
# Create efficiency metrics
stats_df['completion_percentage'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_attempt'] = stats_df['passing_yards'] / (stats_df['attempts'] + 1)
stats_df['yards_per_carry'] = stats_df['rushing_yards'] / (stats_df['carries'] + 1)
stats_df['yards_per_reception'] = stats_df['receiving_yards'] / (stats_df['receptions'] + 1)

# Create touchdown ratios
stats_df['pass_tds_per_attempt'] = stats_df['passing_tds'] / (stats_df['attempts'] + 1)
stats_df['tds_per_carry'] = stats_df['rushing_tds'] / (stats_df['carries'] + 1)
stats_df['tds_per_reception'] = stats_df['receiving_tds'] / (stats_df['receptions'] + 1)

# Calculate season stats for each player over the season
stats_df['season_passing_yards'] = stats_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
stats_df['season_rushing_yards'] = stats_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
stats_df['season_receiving_yards'] = stats_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
stats_df['season_passing_tds'] = stats_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
stats_df['season_rushing_tds'] = stats_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
stats_df['season_receiving_tds'] = stats_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
stats_df['season_completion_percentage'] = stats_df.groupby(['player_id', 'season'])['completion_percentage'].transform('sum')

# Interaction terms
stats_df['completions_per_attempt'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_touch'] = (stats_df['rushing_yards'] + stats_df['receiving_yards']) / (stats_df['carries'] + stats_df['receptions'] + 1) 

In [ ]:
# checking out well known *cough JETS cough* players with essential features
#print(qb_stats_df[qb_stats_df['player_display_name'] == 'Aaron Rodgers'])

#receivers_stats_df[receivers_stats_df['player_display_name'] == 'Garrett Wilson'].head()

#rushers_stats_df[rushers_stats_df['player_display_name'] == 'Breece Hall'].head()


## QBR

not sure if this will even be necessary. might be good to have QBR by season to go along with other QB stats

In [ ]:
qbr_df.info()

In [ ]:
# Define the columns to keep
columns_to_keep = ['season', 'season_type', 'qbr_total', 'name_display']

# Drop columns from qbr_df except the specified ones
qbr_df = qbr_df[columns_to_keep]

# Replace 'Regular' with 'REG' and 'Playoffs' with 'POST' in the 'season_type' column
qbr_df['season_type'] = qbr_df['season_type'].replace({'Regular': 'REG', 'Playoffs': 'POST'})

# Display the modified qbr_df
qbr_df.head()

merge after seperating player positions 

In [ ]:
# Assuming stats_df and qbr_df are your DataFrames
# Define the mapping between columns in stats_df and qbr_df
#column_mapping = {'player_display_name': 'name_display', 'season': 'season', 'season_type': 'season_type'}

# Merge DataFrames using the mapped columns
#qb_stats_df = pd.merge(qb_stats_df, qbr_df, left_on=['player_display_name', 'season', 'season_type'], 
 #                    right_on=['name_display', 'season', 'season_type'], how='left')

# Drop the redundant 'name_display' column
#stats_df.drop(columns=['name_display'], inplace=True)

# Display the merged DataFrame
#stats_df.head()